In [133]:
from datetime import datetime, date, timedelta, time
import pandas as pd
import numpy as np
import re
#import logging
#import json
#import sqlite3
import yaml
import itertools
import os
import pytz
import hashlib
from shutil import copyfile

In [2]:
url='url'
filepath='C:/Users/Neon/Desktop/GitHub/download_entsoe/2017_2_AggregatedGenerationPerType.csv'


In [18]:
def read_entsoe_transparency(filepath, url):
    
    headers = ['region', 'variable', 'attribute', 
               'source', 'subsource','region_type','web']
    
    df = pd.read_csv(
        filepath,
        sep='\t',
        encoding='utf-16',
        header=0,
        index_col=['DateTime'],
        parse_dates=True,
        converters={'MapCode': lambda x: x[:2]},
        #nrows=20,
        )
    
    df.drop(['year', 'month', 'day', 'AreaTypeCode', 'SubmissionTS'], 
            axis=1, inplace=True)
    
    df['AreaName']=[x.replace(' CA','').replace(' BZ','')
                    for x in df['AreaName']]
    
    df.rename(columns ={
        'ActualConsumption': 'consumption', 
        'ActualGenerationOutput': 'generation'},inplace=True)
    
    df.set_index(
        ['MapCode','ProductionType_Name','AreaName','AreaTypeCode'],
        append=True, inplace=True)
    
    df=df.unstack('MapCode')
    df=df.unstack('ProductionType_Name')
    df=df.unstack('AreaName')
    df=df.unstack('AreaTypeCode')
    
    df=pd.concat(
        [df], 
        keys=['ENTSO-E'], 
        names=['source', 'attribute', 'region', 'variable', 'subsource'],
        axis=1)
    df=pd.concat([df], keys= url, names=['web'], axis=1)
    
    df=df.reorder_levels(headers, axis=1)
    
    return df

In [225]:
df = pd.read_csv(
    filepath,
    sep='\t',
    encoding='utf-16',
    header=0,
    index_col=['DateTime'],
    parse_dates=True,
    converters={#'MapCode': lambda y: y[:2],
                #'AreaName': (lambda x: x.replace(' CA','')),
                #'AreaName': map(lambda x: x.replace(' CA','').rstrip(' BZ')),
                #'AreaName': lambda z: z.replace(' BZ',''),
                },
    nrows=500,
    )

#df['AreaName'].replace(' CA','',inplace=True)
#df['AreaName'].re.sub(' CA','')
#df['AreaName'] = df['AreaName'].map(lambda x: x.rstrip(' CA').rstrip(' BZ'))
#df['AreaName'] = df['AreaName'].map(lambda x: x.replace(' CA',''))
#df['AreaName'] = df['AreaName'].map(lambda x: x.replace(' BZ',''))
#df['AreaName']=[x.replace(' CA','').replace(' BZ','') for x in list(df['AreaName'])]
#df['AreaName']=pd.Series([x.replace(' CA','').replace(' BZ','') for x in list(df['AreaName'])], index=df.index)
df['AreaName']=[x.replace(' CA','').replace(' BZ','') for x in df['AreaName']]

df.drop(['year', 'month', 'day'], 
        axis=1, inplace=True)
df.rename(columns ={
    'ActualConsumption': 'consumption', 
    'ActualGenerationOutput': 'generation'},inplace=True)
df.set_index(
    ['MapCode','ProductionType_Name','AreaName','AreaTypeCode'],
    append=True, inplace=True)
df=df.unstack('MapCode')
df=df.unstack('ProductionType_Name')
df=df.unstack('AreaName')
df=df.unstack('AreaTypeCode')
df=pd.concat(
    [df], 
    keys=['ENTSO-E'], 
    names=['source', 'attribute', 'region', 'variable', 'subsource','region_type'],
    axis=1)
df=pd.concat([df], keys= url, names=['web'], axis=1)
df=df.reorder_levels(headers, axis=1)

In [226]:
df.head()

region              DE_TenneT_GER                                     \
variable                 Biomass                                       
attribute             consumption                                      
source                    ENTSO-E                                      
subsource                 Elering     Fingrid     IPTO     MAVIR       
region_type                   BZN CTA     BZN CTA  BZN CTA   BZN CTA   
web                             u   u       u   u    u   u     u   u   
DateTime                                                               
2017-02-01 01:45:00           NaN NaN     NaN NaN  NaN NaN   NaN NaN   
2017-02-01 02:15:00           NaN NaN     NaN NaN  NaN NaN   NaN NaN   
2017-02-01 03:00:00           NaN NaN     NaN NaN  NaN NaN   NaN NaN   
2017-02-01 04:15:00           NaN NaN     NaN NaN  NaN NaN   NaN NaN   
2017-02-01 05:00:00           NaN NaN     NaN NaN  NaN NaN   NaN NaN   

region                                 ...             SK                      \
variable                               ...  Wind Onshore                        
attribute                              ...   SubmissionTS                       
source                                 ...        ENTSO-E                       
subsource           National Grid      ...          MAVIR       National Grid   
region_type                   BZN CTA  ...            BZN   CTA           BZN   
web                             u   u  ...              u     u             u   
DateTime                               ...                                      
2017-02-01 01:45:00           NaN NaN  ...           None  None          None   
2017-02-01 02:15:00           NaN NaN  ...           None  None          None   
2017-02-01 03:00:00           NaN NaN  ...           None  None          None   
2017-02-01 04:15:00           NaN NaN  ...           None  None          None   
2017-02-01 05:00:00           NaN NaN  ...           None  None          None   

region                                                                   
variable                                                                 
attribute                                                                
source                                                                   
subsource                  SEPS       TenneT GER       TransnetBW        
region_type           CTA   BZN   CTA        BZN   CTA        BZN   CTA  
web                     u     u     u          u     u          u     u  
DateTime                                                                 
2017-02-01 01:45:00  None  None  None       None  None       None  None  
2017-02-01 02:15:00  None  None  None       None  None       None  None  
2017-02-01 03:00:00  None  None  None       None  None       None  None  
2017-02-01 04:15:00  None  None  None       None  None       None  None  
2017-02-01 05:00:00  None  None  None       None  None       None  None  

[5 rows x 7680 columns]

In [213]:
df = pd.read_csv(
    filepath,
    sep='\t',
    encoding='utf-16',
    header=0,
    index_col=['DateTime'],
    parse_dates=True,
    converters={#'MapCode': lambda y: y[:2],
                #'AreaName': (lambda x: x.replace(' CA','')),
                #'AreaName': map(lambda x: x.replace(' CA','').rstrip(' BZ')),
                #'AreaName': lambda z: z.replace(' BZ',''),
                #'AreaName': map(lambda x: x.replace(' CA','').replace(' BZ',''))
                },
    nrows=500,
    )

In [216]:
df = pd.read_csv(
    filepath,
    sep='\t',
    encoding='utf-16',
    header=0,
    index_col=['DateTime'],
    parse_dates=True,
    converters={#'MapCode': lambda y: y[:2],
                #'AreaName': (lambda x: x.replace(' CA','')),
                #'AreaName': map(lambda x: x.replace(' CA','').rstrip(' BZ')),
                #'AreaName': lambda z: z.replace(' BZ',''),
                },
    nrows=500,
    )

#df['AreaName'].replace(' CA','',inplace=True),
#df['AreaName'].re.sub(' CA',''),
#df['AreaName'] = df['AreaName'].map(lambda x: x.rstrip(' CA').rstrip(' BZ')),
#df['AreaName'] = df['AreaName'].map(lambda x: x.replace(' CA','')),
#df['AreaName'] = df['AreaName'].map(lambda x: x.replace(' BZ','')),
#df['AreaName']=[x.replace(' CA','').replace(' BZ','') for x in list(df['AreaName'])],
df['AreaName']=pd.Series([x.replace(' CA','').replace(' BZ','') for x in list(df['AreaName'])], index=df.index)


In [145]:
#df['AreaName'].replace(' CA','',inplace=True),
#df['AreaName']= list(df['AreaName'].map(lambda x: x.replace(' CA','').replace(' BZ',''))),
#df['AreaName'].apply(lambda x: x.rstrip(' CA')),
#len(df['AreaName'].map(lambda x: x.replace(' CA',''))),
#df['AreaName'] = df['AreaName'].map(lambda x: x.replace(' BZ','')),
#df['MapCode'] = df['MapCode'].map(lambda y: y[:2]),
#df['MapCode'] = [w.replace(' CA','') for w in df['MapCode']],
#df['MapCode'] = map(lambda x: str.replace(x, ' CA',''), list(df['MapCode']))

(500,)

In [143]:
#s=list(s)
#len(df['AreaName'].map(lambda x: re.findall(" CA", x)))
#len(df['AreaName'].index)

500

In [201]:
df['AreaName'].update(pd.Series([x.replace(' CA','').replace(' BZ','') for x in df['AreaName']])),

(None,)

In [214]:
df['AreaName']=pd.Series([x.replace(' CA','').replace(' BZ','') for x in list(df['AreaName'])], index=df.index)
#df['AreaName']=[x.replace(' CA','').replace(' BZ','') for x in df['AreaName']]


In [215]:
#df.head()
df['AreaName']

DateTime
2017-02-01 12:15:00         MAVIR
2017-02-01 12:15:00         MAVIR
2017-02-01 18:15:00         MAVIR
2017-02-01 10:00:00         MAVIR
2017-02-01 11:45:00         MAVIR
2017-02-23 03:00:00    TransnetBW
2017-02-23 15:00:00    TransnetBW
2017-02-23 16:45:00    TransnetBW
2017-02-23 09:15:00    TransnetBW
2017-02-23 00:30:00    TransnetBW
2017-02-23 01:30:00    TransnetBW
2017-02-23 16:30:00    TransnetBW
2017-02-10 09:30:00    TransnetBW
2017-02-10 04:45:00    TransnetBW
2017-02-10 15:45:00    TransnetBW
2017-02-10 18:00:00    TransnetBW
2017-02-10 01:00:00    TransnetBW
2017-02-10 01:00:00    TransnetBW
2017-02-10 22:00:00    TransnetBW
2017-02-10 17:30:00    TransnetBW
2017-02-10 01:45:00    TransnetBW
2017-02-10 06:00:00    TransnetBW
2017-02-10 01:30:00    TransnetBW
2017-02-10 07:15:00    TransnetBW
2017-02-10 14:00:00    TransnetBW
2017-02-10 06:30:00    TransnetBW
2017-02-09 02:00:00    TransnetBW
2017-02-09 18:00:00    TransnetBW
2017-02-09 07:15:00    TransnetBW
2017-